In [7]:
from config import COMMON_CONFIG

In [8]:
!pip install matplotlib -q
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

!pip install pandas -q
import pandas as pd
# Хак чтобы работало на Anaconda под MacOs 10.15.7 Catalina
pd.DataFrame.iteritems = pd.DataFrame.items

In [9]:
#!pip install git+https://github.com/kernc/backtesting.py.git -q
!pip install backtesting -q

from backtesting import Strategy
from backtesting_utils import run_backtest
from backtesting_utils import convert_data_to_bt_format

In [10]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

import random
import numpy as np

def seed_everything(seed: int = COMMON_CONFIG.SEED) -> None:
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    print(f"Using {seed} seed")

# Зафиксируем SEED
seed_everything(COMMON_CONFIG.SEED)

Using 777 seed


In [11]:
from file_utils import read_data_from_file
from data_utils import train_test_split

df = read_data_from_file(COMMON_CONFIG.SYMBOL, COMMON_CONFIG.TIMEFRAME)
df

,open,high,low,close,volume
date,,,,,
2020-01-01 00:00:00,129.16,129.19,128.68,128.87,7769.17336
2020-01-01 01:00:00,128.87,130.65,128.78,130.64,11344.65516
2020-01-01 02:00:00,130.63,130.98,130.35,130.85,7603.35623
2020-01-01 03:00:00,130.85,130.89,129.94,130.20,4968.55433
2020-01-01 04:00:00,130.21,130.74,130.15,130.20,3397.90747
...,...,...,...,...,...
2025-01-19 09:00:00,3221.90,3233.77,3143.02,3226.00,105144.51170
2025-01-19 10:00:00,3226.01,3226.89,3172.97,3174.70,51453.00680
2025-01-19 11:00:00,3174.70,3188.90,3130.48,3149.97,50955.62760


In [12]:
!pip install tsfresh -q

from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series
from tsfresh.utilities.dataframe_functions import impute

In [ ]:
df_for_tsfresh = df.copy()[:10000]
df_for_tsfresh

In [ ]:
df_melted = pd.DataFrame()
df_melted["timestamp"] = df_for_tsfresh.index
df_melted["close"] = df_for_tsfresh['close'].values
df_melted['symbol'] = COMMON_CONFIG.SYMBOL.replace('/', '')
df_melted.tail(3)

In [ ]:
df_rolled = roll_time_series(df_melted, column_id="symbol", column_sort="timestamp",
                             min_timeshift=COMMON_CONFIG.TS_FRESH_MIN_WINDOW_SIZE,
                             max_timeshift=COMMON_CONFIG.TS_FRESH_MAX_WINDOW_SIZE)
df_rolled

In [ ]:
X = extract_features(df_rolled.drop("symbol", axis=1), 
                     column_id="id", column_sort="timestamp", column_value="close", 
                     impute_function=impute, show_warnings=False)
X

In [ ]:
X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
X.index.name = "last_timestamp"
X

In [ ]:
y = df_melted.set_index("timestamp").sort_index().close.shift(-1)
y.tail(3)

In [ ]:
y = y[y.index.isin(X.index)]
X = X[X.index.isin(y.index)]

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_val, X_test = train_test_split(X)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train, y_val, y_test = train_test_split(y)

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
X_train_selected = select_features(X_train, y_train)
X_train_selected

In [ ]:
#X_val_selected = X_val[X_train_selected.columns]
#X_val_selected

In [ ]:
X_test_selected = X_test[X_train_selected.columns]
X_test_selected

In [ ]:
from sklearn.utils import all_estimators
from sklearn import base
import sklearn.metrics as metrics 

from functions import init_regressors,load_data,get_best_regressor,view_performance

# Choose whether to look for a best regression model, or use a pre-saved one from a previous run, alternatively you can choose your own regression method.
init = 0                  # do not (0) or (1) calculate the best regressor
best_regressor = ''       # you can choose a regressor here

# Choose to see the best_regressor calculations in verbose mode
verbose = 0

ticker = COMMON_CONFIG.SYMBOL.replace('/', '')

test_start = X.index.max() - pd.DateOffset(months=int(1))
test_start

In [ ]:
if init == 1:
    init_regressors(ticker)

In [ ]:
# For cleaner output we suppress warnings that occur during testing of the estimators
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning

simplefilter("ignore", category=FutureWarning)
simplefilter("ignore", category=RuntimeWarning)
simplefilter("ignore", category=ConvergenceWarning)
simplefilter("ignore", category=UserWarning)

# Check and proceed if init=1 or no best_regressor was found
if (best_regressor == '') | init == 1:

    # Create a list for regressors with too bad performance
    new_removed_regressors = []
    removed_regressors = pd.DataFrame()

    # Calculate all the estimator metrics
    MAE, MSE, RMSE, R2, removed_regressors, new_removed_regressors = get_best_regressor(X_train_selected, X_test,y_train, y, test_start, ticker, verbose)

    # Write the results to this file
    with open(f'regressor_min_mae_{ticker}.csv', 'w') as f:
        f.write(min(MAE, key=MAE.get))

    # Define the best estimator by chosing the one with the smallest MAE
    best_regressor = min(MAE, key=MAE.get)

    # Create a dataframe with the excluded estimators and write it to a file, so we can exclude them in future runs 
    if new_removed_regressors != 'None':
        new_removed_regressors = pd.DataFrame(new_removed_regressors, columns=['name'])
        removed_regressors = pd.concat([removed_regressors, new_removed_regressors])
        removed_regressors.to_csv(f'removed_regressors_{ticker}.csv', 'a', index=False)

# Load the estimator model and store its name
estimators = all_estimators()
for est_name, estimator in estimators:
    if str(best_regressor) == est_name:
        best_regressor = estimator()
        name = est_name

# Print out the five best estimators
df_best_regressors = pd.DataFrame.from_dict({'MAE': MAE, 'RMSE': RMSE, 'R2_score': R2})
df_best_regressors = df_best_regressors.sort_values(by="MAE", ascending=True).head().reset_index()
df_styled = df_best_regressors.style.apply(lambda x: ['background-color: lightgreen' if x.name == 0 else '' for i in x],
                                           axis=1)

display(df_styled)

In [ ]:
try:
    with open(f'regressor_min_mae_{ticker}.csv') as f:
        best_regressor=f.read()
        # Load the estimator model and store its name
        estimators=all_estimators()
        for est_name, estimator in estimators:    
            if str(best_regressor)==est_name:
                best_regressor=estimator()
                name=est_name
    print ('Last saved regression method:',best_regressor)
except:
    pass

In [ ]:
best_regressor.fit(X_train_selected, y_train)

In [ ]:
y_pred = pd.DataFrame(best_regressor.predict(X_test_selected), index=X_test_selected.index, columns=['predicted_close'])
y_pred

In [ ]:
y_test.tail(3)

In [ ]:
y_pred.tail(3)

In [ ]:
from plots import plot

all_predictions = y_pred#['predicted_close']
all_targets = y_test#['close']
plot(all_predictions, all_targets, "ML model Predictions vs. Actual")

In [ ]:
df_best_regressors